In [1]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
from scipy.optimize import minimize
from utility_code.utility import utils

In [2]:
#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")

supp = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')
supp = supp[supp['Location'].isin(train['Province_State'])]
supp['Population'] = supp['Number of COVID-19 Cases'].divide(supp['COVID-19 Cases per 1,000,000 Population']) * 1e6

states = pd.Series.unique(train['Province_State'])
num_states = len(states)

In [3]:
#only look at the features in features list
features = ['Confirmed','Active']
num_features = len(features)

#stratify by state (into state dictionary)
statesdata = {}
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s,features]

WINDOW_SIZE = 4
state_feature_indices = utils.get_column_indices(statesdata['California'],features)

#append the feature spaces from the W days prior (where W is the window length)
new_features = []
for day in range(WINDOW_SIZE):
    for f in features:
        new_features.append(f + "(-"+ str(WINDOW_SIZE-day) + " days)")
all_new_features = new_features + features


In [4]:
# x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
# y = knndata[['Confirmed', 'Deaths']]

In [5]:
# toguess = 1
# trainx = x.head(len(x)-toguess)
# testx = x.tail(toguess)
# trainy = y.head(len(y)-toguess)
# testy = y.tail(toguess)

In [6]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor


#from sklearn.ensemble import StackingRegressor
#estimators = [('ridge', Ridge(random_state = 123)), ('lasso', LassoCV(random_state=445)), 
              #('svr', SVR(kernel='linear',C=1, gamma=1e-6))]
reg = MultiOutputRegressor(estimator=Ridge(alpha=100,max_iter=10000,tol=0.0005))
#reg = MultiOutputRegressor(estimator=ElasticNet(max_iter=2000, l1_ratio=0.1))
#reg = MultiOutputRegressor(estimator=BayesianRidge(alpha_1=1.5, alpha_2=1e-2, lambda_1=1.5, lambda_2 = 1e-2))
#reg = MultiOutputRegressor(estimator=BaggingRegressor(Ridge(alpha = 100)))

#from sklearn.svm import LinearSVR
#reg = MultiOutputRegressor(estimator=LinearSVR(max_iter=2000))
# reg.fit(trainx, trainy)
# reg.predict(testx)

In [7]:
# states

# days_to_predict = 26
# for d in range(days_to_predict):
#     x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
#     y = knndata[['Confirmed', 'Deaths']]
#     toguess = 1
#     trainx = x.head(len(x))
#     trainy = y.head(len(y))
#     # testy = y.tail(toguess)

#     reg.fit(trainx, trainy)
    

#     ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
#     #ftrs.drop(columns=knndata.columns[[0,1]])
   
#     ftrs.columns = knndata.columns[0:2*WINDOW_SIZE]
    
#     new = reg.predict(ftrs)
#     ftrs = np.append(ftrs, new)
#     ftrs = ftrs.astype(int)
#     if d==0: print(ftrs)
#     knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)
    
#     # append to knndata
#     if d == 0: 
#         print(knndata)
# done = knndata.tail(days_to_predict)
# done = done[['Confirmed', 'Deaths']]
# print(done)

In [8]:
proj = {}

for s in states:
    
    a = statesdata[s]

    #fill the knn data using days from training set
    knndata = pd.DataFrame(columns = all_new_features)
    num_training_days = len(statesdata['California'])

    #fill the table
    for d in range(WINDOW_SIZE,num_training_days):
        knndata_row_index = knndata.shape[0]
        knn_row = utils.flatten_dataframe(a,slice(d-WINDOW_SIZE,d+1), state_feature_indices)
        utils.dataframe_append_row(knndata,knn_row,s,d)   
    

    days_to_predict = 26
    for d in range(days_to_predict):
        #x = knndata.drop(columns=features)
        #y = knndata.drop(columns=features)
        x = knndata.drop(['Confirmed', 'Active'], axis = 1)
        y = knndata[['Confirmed', 'Active']]
        toguess = 1
        trainx = x.head(len(x))
        trainy = y.head(len(y))
        # testy = y.tail(toguess)
        #reg = MultiOutputRegressor(estimator=Ridge(random_state=123, max_iter=1500))
        reg.fit(trainx, trainy)

        #rmv = [i for i in range(num_features)]
        #ftrs = knndata.drop(columns=knndata.columns[rmv]).tail(1)
        ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
        #ftrs.drop(columns=knndata.columns[[0,1]])

        ftrs.columns = knndata.columns[0:num_features*WINDOW_SIZE]

        new = reg.predict(ftrs)
        ftrs = np.append(ftrs, new)
        ftrs = ftrs.astype(int)
        #if d==0: print(ftrs)
        knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)

        # append to knndata
        #if d == 0: 
            #print(knndata)
    done = knndata.tail(days_to_predict)
    done = done[['Confirmed', 'Active']]
    #print(done)
    proj[s] = done

In [9]:
order = test.loc[0:49,'Province_State']
test

# format submission
conf = []
ir = []
fid = 0
for i in range(days_to_predict):
    for j in order:
        projection = proj[j].iloc[i]
        print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        ir.append(int(projection['Active']))
        print(fid)
        fid+=1
    
    

Alabama day 0
0
Alaska day 0
1
Arizona day 0
2
Arkansas day 0
3
California day 0
4
Colorado day 0
5
Connecticut day 0
6
Delaware day 0
7
Florida day 0
8
Georgia day 0
9
Hawaii day 0
10
Idaho day 0
11
Illinois day 0
12
Indiana day 0
13
Iowa day 0
14
Kansas day 0
15
Kentucky day 0
16
Louisiana day 0
17
Maine day 0
18
Maryland day 0
19
Massachusetts day 0
20
Michigan day 0
21
Minnesota day 0
22
Mississippi day 0
23
Missouri day 0
24
Montana day 0
25
Nebraska day 0
26
Nevada day 0
27
New Hampshire day 0
28
New Jersey day 0
29
New Mexico day 0
30
New York day 0
31
North Carolina day 0
32
North Dakota day 0
33
Ohio day 0
34
Oklahoma day 0
35
Oregon day 0
36
Pennsylvania day 0
37
Rhode Island day 0
38
South Carolina day 0
39
South Dakota day 0
40
Tennessee day 0
41
Texas day 0
42
Utah day 0
43
Vermont day 0
44
Virginia day 0
45
Washington day 0
46
West Virginia day 0
47
Wisconsin day 0
48
Wyoming day 0
49
Alabama day 1
50
Alaska day 1
51
Arizona day 1
52
Arkansas day 1
53
California day 1
54


Hawaii day 12
610
Idaho day 12
611
Illinois day 12
612
Indiana day 12
613
Iowa day 12
614
Kansas day 12
615
Kentucky day 12
616
Louisiana day 12
617
Maine day 12
618
Maryland day 12
619
Massachusetts day 12
620
Michigan day 12
621
Minnesota day 12
622
Mississippi day 12
623
Missouri day 12
624
Montana day 12
625
Nebraska day 12
626
Nevada day 12
627
New Hampshire day 12
628
New Jersey day 12
629
New Mexico day 12
630
New York day 12
631
North Carolina day 12
632
North Dakota day 12
633
Ohio day 12
634
Oklahoma day 12
635
Oregon day 12
636
Pennsylvania day 12
637
Rhode Island day 12
638
South Carolina day 12
639
South Dakota day 12
640
Tennessee day 12
641
Texas day 12
642
Utah day 12
643
Vermont day 12
644
Virginia day 12
645
Washington day 12
646
West Virginia day 12
647
Wisconsin day 12
648
Wyoming day 12
649
Alabama day 13
650
Alaska day 13
651
Arizona day 13
652
Arkansas day 13
653
California day 13
654
Colorado day 13
655
Connecticut day 13
656
Delaware day 13
657
Florida day 13
6

Hawaii day 22
1110
Idaho day 22
1111
Illinois day 22
1112
Indiana day 22
1113
Iowa day 22
1114
Kansas day 22
1115
Kentucky day 22
1116
Louisiana day 22
1117
Maine day 22
1118
Maryland day 22
1119
Massachusetts day 22
1120
Michigan day 22
1121
Minnesota day 22
1122
Mississippi day 22
1123
Missouri day 22
1124
Montana day 22
1125
Nebraska day 22
1126
Nevada day 22
1127
New Hampshire day 22
1128
New Jersey day 22
1129
New Mexico day 22
1130
New York day 22
1131
North Carolina day 22
1132
North Dakota day 22
1133
Ohio day 22
1134
Oklahoma day 22
1135
Oregon day 22
1136
Pennsylvania day 22
1137
Rhode Island day 22
1138
South Carolina day 22
1139
South Dakota day 22
1140
Tennessee day 22
1141
Texas day 22
1142
Utah day 22
1143
Vermont day 22
1144
Virginia day 22
1145
Washington day 22
1146
West Virginia day 22
1147
Wisconsin day 22
1148
Wyoming day 22
1149
Alabama day 23
1150
Alaska day 23
1151
Arizona day 23
1152
Arkansas day 23
1153
California day 23
1154
Colorado day 23
1155
Connecticut d

In [20]:
test['Confirmed'] = conf
test['Active'] = ir

In [21]:
submission = test.drop(columns=['Date', 'Province_State'])
print(submission)

      ForecastID  Confirmed  Deaths  Incident_Rate  Active
0              0     127340      -1          76089   76089
1              1       5311      -1           2993    2993
2              2     202186      -1         166285  166285
3              3      61718      -1           5366    5366
4              4     720284      -1         707121  707121
...          ...        ...     ...            ...     ...
1295        1295     146831      -1         125100  125100
1296        1296      92771      -1          90583   90583
1297        1297      14886      -1           2775    2775
1298        1298      92787      -1           6421    6421
1299        1299       5097      -1            759     759

[1300 rows x 5 columns]


In [12]:
#submission.to_csv("team31-dec1-1.csv", index=False, header=True)

In [13]:
trainrd2 = pd.read_csv("train_round2.csv")


In [14]:
datestofind = trainrd2['Date'] == '09-01-2020'
start = datestofind[datestofind == True].index[0]
datestofind2 = trainrd2['Date'] == '09-26-2020'
end = datestofind[datestofind == True].index[49]

validation = trainrd2.iloc[start:end,:]
validation

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
7100,7100,Alabama,09-01-2020,127616,2200,48028.0,77388.0,2602.716398,958920.0,NaN,1.723922,19557.083814,NaN
7101,7101,Alaska,09-01-2020,5297,39,2246.0,3012.0,724.083959,374255.0,NaN,0.736266,51159.532223,NaN
7102,7102,Arizona,09-01-2020,202342,5044,30841.0,166457.0,2779.912998,1208990.0,NaN,2.492809,16609.932767,NaN
7103,7103,Arkansas,09-01-2020,61497,814,55647.0,5036.0,2037.806299,731025.0,NaN,1.323642,24223.740177,NaN
7104,7104,California,09-01-2020,715617,13150,NaN,702467.0,1811.128167,11470696.0,NaN,1.837575,29030.753344,NaN
7105,7105,Colorado,09-01-2020,57755,1946,5953.0,49856.0,1002.911055,1009970.0,NaN,3.369405,17538.050017,NaN
7106,7106,Connecticut,09-01-2020,53006,4466,8967.0,39573.0,1486.724631,1192338.0,NaN,8.425461,33442.973876,NaN
7107,7107,Delaware,09-01-2020,17535,605,9419.0,7511.0,1800.744328,238423.0,NaN,3.450242,24484.680066,NaN
7108,7108,Florida,09-01-2020,631040,11374,NaN,619666.0,2938.112148,4675866.0,NaN,1.802421,21770.757320,NaN
7109,7109,Georgia,09-01-2020,272697,5733,NaN,266964.0,2568.391596,2368498.0,NaN,2.102333,22307.654127,NaN


In [15]:
def MAPE(pred, valid):
    pred = pred.reset_index()
    valid = valid.reset_index()
    pred = pred.astype('int64')
    valid = valid.astype('int64')
    v = pred.subtract(valid)
    v = v.divide(valid)
    v = v.abs()
    v = v.sum(axis = 0)
    #v = v[0]+v[1]+v[2]
    n = len(pred)
    return v/n

In [16]:
bestsub = pd.read_csv("team31-nov29-2.csv")

In [17]:
#MAPE(bestsub[['Confirmed', 'Incident_Rate']], validation[['Confirmed', 'Incident_Rate']])*len(test)

In [22]:
MAPE(submission[['Confirmed', 'Active']], validation[['Confirmed', 'Active']])

index        0.037565
Confirmed    0.000089
Active       0.000736
dtype: float64

In [19]:
submission

,ForecastID,Confirmed,Deaths,Incident_Rate
0,0,127340,-1,76089
1,1,5311,-1,2993
2,2,202186,-1,166285
3,3,61718,-1,5366
4,4,720284,-1,707121
...,...,...,...,...
1295,1295,146831,-1,125100
1296,1296,92771,-1,90583
1297,1297,14886,-1,2775
1298,1298,92787,-1,6421
